 - Objective

Objective is to draw insights from the restaurant data for food delivery service.

Some ideas of conclusions could include: trying to identify the trending restaurants with your own scoring algorithm (can be as simple as the best rating or most votes or both!), clustering restaurant locations to figure out the optimized FoodieX pick up zones, estimating cook time based on restaurant info, and demonstrating your findings using a data visualization tool. As an important member of the FoodieX team, you get to come up with your own analysis! So try your best to dig out any useful information out of this data set. The sky's the limit! 🤓!

 - Data modification

In [141]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt

In [142]:
RestaurantData = pd.read_csv('2020-XTern-DS.csv')
# RestaurantData.head(50)

In [171]:
RestaurantData.head()

,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time,...,Naga,Korean,Modern Indian,Awadhi,Kashmiri,French,Portuguese,Charcoal Chicken,Frozen Yogurt,Bangladeshi
0,ID_6321,39.262605,-85.837372,"Fast Food, Rolls, Burger, Salad, Wraps",20.0,50.0,3.5,12.0,4.0,30.0,...,0,0,0,0,0,0,0,0,0,0
1,ID_2882,39.775933,-85.740581,"Ice Cream, Desserts",10.0,50.0,3.5,11.0,4.0,30.0,...,0,0,0,0,0,0,0,0,0,0
2,ID_1595,39.253436,-85.123779,"Italian, Street Food, Fast Food",15.0,50.0,3.6,99.0,30.0,65.0,...,0,0,0,0,0,0,0,0,0,0
3,ID_5929,39.029841,-85.332050,"Mughlai, North Indian, Chinese",25.0,99.0,3.7,176.0,95.0,30.0,...,0,0,0,0,0,0,0,0,0,0
4,ID_6123,39.882284,-85.517407,"Cafe, Beverages",20.0,99.0,3.2,521.0,235.0,65.0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
RestaurantData.shape      

(2019, 10)

In [144]:
RestaurantData.dtypes

Restaurant        object
Latitude         float64
Longitude        float64
Cuisines          object
Average_Cost      object
Minimum_Order     object
Rating            object
Votes             object
Reviews           object
Cook_Time         object
dtype: object

In [182]:
a = RestaurantData.groupby('Restaurant').Restaurant.count()
len(a)
a[5]

1

In [184]:
for i in range(len(a)):
    if a[i] > 1:
        print(a.index[i], 'has', a[i],'observations')

    

ID_1004 has 2 observations
ID_1074 has 2 observations
ID_1140 has 3 observations
ID_1156 has 3 observations
ID_1166 has 2 observations
ID_1175 has 3 observations
ID_1204 has 2 observations
ID_1209 has 4 observations
ID_1270 has 2 observations
ID_147 has 2 observations
ID_1521 has 2 observations
ID_1532 has 2 observations
ID_1555 has 2 observations
ID_1631 has 2 observations
ID_1721 has 2 observations
ID_1784 has 2 observations
ID_1796 has 3 observations
ID_18 has 2 observations
ID_1867 has 2 observations
ID_188 has 5 observations
ID_1902 has 2 observations
ID_1966 has 2 observations
ID_2034 has 2 observations
ID_207 has 2 observations
ID_2083 has 2 observations
ID_2116 has 2 observations
ID_2132 has 2 observations
ID_2301 has 4 observations
ID_2356 has 2 observations
ID_2376 has 2 observations
ID_2402 has 3 observations
ID_2483 has 5 observations
ID_2542 has 2 observations
ID_2595 has 2 observations
ID_2648 has 2 observations
ID_2678 has 2 observations
ID_2718 has 3 observations
ID_273

In [186]:
RestaurantData.loc[RestaurantData.Restaurant == 'ID_188']

,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time,...,Naga,Korean,Modern Indian,Awadhi,Kashmiri,French,Portuguese,Charcoal Chicken,Frozen Yogurt,Bangladeshi
206,ID_188,39.505842,-85.643108,"North Indian, Mughlai",15.0,50.0,3.3,4.0,2.0,30.0,...,0,0,0,0,0,0,0,0,0,0
361,ID_188,39.829219,-85.855595,"North Indian, Mughlai",15.0,50.0,2.8,0.0,0.0,30.0,...,0,0,0,0,0,0,0,0,0,0
1312,ID_188,39.832759,-85.418297,"North Indian, Mughlai",15.0,50.0,3.3,12.0,3.0,30.0,...,0,0,0,0,0,0,0,0,0,0
1806,ID_188,39.589548,-85.077852,"North Indian, Mughlai, Beverages",15.0,50.0,3.2,13.0,2.0,30.0,...,0,0,0,0,0,0,0,0,0,0
1826,ID_188,39.113495,-85.365590,"North Indian, Mughlai",15.0,50.0,2.8,0.0,0.0,30.0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
RestaurantData.Cuisines.unique()

array(['Fast Food, Rolls, Burger, Salad, Wraps', 'Ice Cream, Desserts',
       'Italian, Street Food, Fast Food',
       'Mughlai, North Indian, Chinese', 'Cafe, Beverages',
       'South Indian, North Indian, Chinese', 'Beverages, Fast Food',
       'Chinese, Thai, Asian', 'Mithai, Street Food',
       'Fast Food, North Indian, Rolls, Chinese, Momos, Mughlai',
       'North Indian, Chinese, Mughlai',
       'Chinese, Thai, Indonesian, Italian',
       'Burger, Fast Food, Desserts, Beverages', 'Beverages', 'Fast Food',
       'Biryani, Mughlai', 'Italian',
       'Chinese, North Indian, South Indian, Fast Food', 'Kerala, Bihari',
       'South Indian', 'Chinese, North Indian, Rolls, Momos',
       'North Indian, Rolls', 'North Indian, Chinese',
       'Biryani, North Indian', 'North Indian, Chinese, Fast Food',
       'Desserts, Ice Cream', 'Fast Food, South Indian, Street Food',
       'North Indian', 'Cafe, Mexican, Japanese, Italian, Chinese',
       'Bakery, Fast Food, Rolls, Bever

In [147]:
category = []
for i in range(RestaurantData.shape[0]):
    temp_list  = RestaurantData.Cuisines[i].split(', ')
    for j in temp_list:
        if j not in category:
            category.append(j)

In [148]:
for n in category:
    RestaurantData[n] = 0
for i in range(RestaurantData.shape[0]):
    temp_list  = RestaurantData.Cuisines[i].split(', ')
    for n in category:
        if n in temp_list:
            RestaurantData.loc[i, n] = 1

In [149]:
# RestaurantData.head(10)

In [150]:
RestaurantData.Average_Cost.unique()

array(['$20.00', '$10.00', '$15.00', '$25.00', '$65.00', '$35.00',
       '$80.00', '$5.00', '$40.00', '$60.00', '$30.00', '$75.00',
       '$45.00', '$55.00', '$100.00', '$50.00', '$90.00', '1,00', '1,20',
       '$95.00', '$85.00', '$150.00'], dtype=object)

In [151]:
RestaurantData.Average_Cost = RestaurantData.Average_Cost.apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.float64)

In [152]:
RestaurantData.Average_Cost.describe()

count    2019.000000
mean       20.123824
std        12.986214
min         5.000000
25%        10.000000
50%        20.000000
75%        20.000000
max       150.000000
Name: Average_Cost, dtype: float64

In [153]:
RestaurantData.Minimum_Order.unique()

array(['$50.00', '$99.00', '$0.00', '$200.00', '$450.00', '$350.00',
       '$79.00'], dtype=object)

In [154]:
RestaurantData.Minimum_Order = RestaurantData.Minimum_Order.apply(lambda x: x.replace('$','')).astype(np.float64)

In [155]:
RestaurantData.Minimum_Order.describe()

count    2019.000000
mean       53.342249
std        20.314896
min         0.000000
25%        50.000000
50%        50.000000
75%        50.000000
max       450.000000
Name: Minimum_Order, dtype: float64

In [156]:
RestaurantData.Rating.unique()
RestaurantData.Rating.value_counts()

-               216
3.7             169
3.5             149
3.8             141
NEW             135
3.3             131
3.9             131
3.4             130
3.6             124
4.0             113
4.1              89
3.2              87
3.1              80
4.2              53
3.0              51
4.3              45
2.9              43
2.8              42
4.4              27
2.7              13
4.5              12
4.7              10
4.6               9
2.6               9
2.5               5
Opening Soon      2
4.8               2
2.4               1
Name: Rating, dtype: int64

In [169]:
RestaurantData.Rating = RestaurantData.Rating.apply(lambda x: x.replace('NEW','-')).apply(lambda x: x.replace('Opening Soon','-'))

In [170]:
RestaurantData.Rating.unique()
RestaurantData.Rating.value_counts()

-      353
3.7    169
3.5    149
3.8    141
3.9    131
3.3    131
3.4    130
3.6    124
4.0    113
4.1     89
3.2     87
3.1     80
4.2     53
3.0     51
4.3     45
2.9     43
2.8     42
4.4     27
2.7     13
4.5     12
4.7     10
4.6      9
2.6      9
2.5      5
4.8      2
2.4      1
Name: Rating, dtype: int64

In [157]:
RestaurantData.Votes.unique()
RestaurantData.Votes.value_counts()

-       380
4        45
12       39
7        38
9        37
       ... 
1169      1
2664      1
1349      1
1857      1
527       1
Name: Votes, Length: 524, dtype: int64

In [161]:
RestaurantData.Votes = RestaurantData.Votes.apply(lambda x: x.replace('-','0')).astype(np.float64)

In [162]:
RestaurantData.Reviews.describe()

count    2019.000000
mean      101.192174
std       298.857200
min         0.000000
25%         2.000000
50%        13.000000
75%        69.000000
max      6504.000000
Name: Reviews, dtype: float64

In [163]:
RestaurantData.Reviews.unique()
RestaurantData.Reviews.value_counts()

0.0       415
2.0        78
1.0        71
3.0        70
5.0        62
         ... 
565.0       1
1150.0      1
518.0       1
938.0       1
262.0       1
Name: Reviews, Length: 360, dtype: int64

In [164]:
RestaurantData.Reviews = RestaurantData.Reviews.apply(lambda x: x.replace('-','0')).astype(np.float64)

AttributeError: 'float' object has no attribute 'replace'

In [165]:
RestaurantData.Reviews.describe()

count    2019.000000
mean      101.192174
std       298.857200
min         0.000000
25%         2.000000
50%        13.000000
75%        69.000000
max      6504.000000
Name: Reviews, dtype: float64

In [166]:
RestaurantData.Cook_Time.unique()

array(['30 minutes', '65 minutes', '45 minutes', '10 minutes',
       '20 minutes', '120 minutes', '80 minutes'], dtype=object)

In [167]:
RestaurantData.Cook_Time = RestaurantData.Cook_Time.apply(lambda x: x.replace(' minutes','')).astype(np.float64)

In [168]:
RestaurantData.Cook_Time.describe()

count    2019.000000
mean       36.921743
std        11.804352
min        10.000000
25%        30.000000
50%        30.000000
75%        45.000000
max       120.000000
Name: Cook_Time, dtype: float64

- Conclusions

- Rating prediction for new restaurants based on restaurant info, by random forest

- Clustering restaurant locations to figure out the optimized FoodieX pick up zones